----
<div style="display: flex; align-items: center;">
    <img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="20%" style="vertical-align: top;">
    <h1 style="margin-left: 20px;">Data Library for Python</h1>
</div>

----

## Content - Pricing - Chain + Streaming Constituents

This notebook demonstrates how to use the Pricing interface to 
- retrieve the constituents of a Chain instrument
- and then request Streaming data for the Constituents themselves

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the LD Library in the **_lseg-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _LD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["LD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [2]:
from lseg.data.content import pricing
import lseg.data as ld
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [3]:
ld.open_session()

<lseg.data.session.Definition object at 0x116df5ac0 {name='workspace'}>

### Define callbacks to display the streaming Constituents data
We will use these later to display the streaming data for the Constituent RICs

In [4]:
df = DataFrame()

# Function to initially populate Dataframe, once initial values received for all items
def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)

# Function to update dataframe, when we receive updates for individual items
def update_dataframe(fields, instrument_name, streaming_prices):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)

### Get the Chain's constituent RICs
Define & snap the current constituents of the Chain and then display them

In [5]:
# Define a Chain object for the Dow Jones Index
dow = pricing.chain.Definition(name="0#.DJI").get_stream()
# We just want a snapshot of the current Index
dow.open(with_updates=False)
## Extract the consituent RICs and display
dow_constituents = dow.constituents
print(dow_constituents)

['AAPL.OQ', 'AMGN.OQ', 'AMZN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WMT.N']


## Create Streams for the consitutuents
Define streams for all the consituents of the above chain, specifying two fields.   
Specify callbacks to display the field values:
- Call *create_dataframe* once we can received the intial response for all the constituents
- Call *update_dataframe* as and when we receive field update for any of the consituents

In [6]:
# Define our Streaming Price object for all the consituents
pricing_stream = pricing.Definition(universe=dow_constituents, fields=["BID", "ASK"]).get_stream()
# Specify the callback to create and display a dataframe 
# on_complete is invoked once we receive the initial image for all our instruments
pricing_stream.on_complete(create_dataframe)
#Specify the callback to display subsequent updates as and when received
pricing_stream.on_update(update_dataframe)


<lseg.data.content.pricing.Stream object at 0x10329dc70 {name='['AAPL.OQ', 'AMGN.OQ', 'AMZN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WMT.N']'}>

## Open the streams for all our constituents

In [7]:
# Request all instrument from server and open streams
pricing_stream.open()
# The callbacks we declared earlier should be invoked as and when we receive server responses
# i.e. create_dataframe will be called once we have the initial values for each instrument
# after which, update_dataframe will be invoked as and when any instrument's BID/ASK fields are updated

ASK     BID
AAPL.OQ  214.15     214
AMGN.OQ  332.79  298.88
AMZN.OQ  182.89  182.74
AXP.N         0       0
BA.N          0       0
CAT.N         0       0
CRM.N         0       0
CSCO.OQ      46   45.85
CVX.N         0       0
DIS.N         0       0
DOW.N         0       0
GS.N          0       0
HD.N          0       0
HON.OQ    213.5     197
IBM.N         0       0
INTC.OQ   30.64   30.59
JNJ.N         0       0
JPM.N         0       0
KO.N          0       0
MCD.N         0       0
MMM.N         0       0
MRK.N         0       0
MSFT.OQ  446.78  446.36
NKE.N         0       0
PG.N          0       0
TRV.N         0       0
UNH.N         0       0
V.N           0       0
VZ.N          0       0
WMT.N         0       0

<OpenState.Opened: 'Opened'>

## Close the streams for the constituents

In [8]:
pricing_stream.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [9]:
ld.close_session()